In [9]:
from keras.datasets import mnist
import numpy as np
from numpy import array
from math import log
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from itertools import product
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn.svm import SVC
from sklearn.feature_selection import *
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import accuracy_score
import tensorflow as tf
import copy
from skimage.transform import resize
from skimage.util import *

#load MNIST
(X_train_raw, y_train_raw), (X_test_raw, y_test_raw) = mnist.load_data()

In [10]:
def image_set_preprocessing(X, y, batch_ratio = 1, pad_size=2, pad_method='constant'):
    X_pad = np.pad(X, ((0,0),(pad_size,pad_size),(pad_size,pad_size)), 'constant')
    
    batch_size = (int)(X_pad.shape[0]*batch_ratio)
    
    #order = np.array(range(X_pad.shape[0]))
    #np.random.shuffle(order)
    #X_pad_shuffle = X_pad[order]
    #y_shuffle = y[order]
    X_pad_shuffle = X_pad
    y_shuffle = y

    X_train_batch = ((X_pad_shuffle[0:batch_size, :, :]).astype('float32'))/255
    y_train_batch = y_shuffle[0:batch_size,]
    X_train_batch = X_train_batch.reshape(X_train_batch.shape[0], X_train_batch.shape[1], X_train_batch.shape[2], 1)
    
    return X_train_batch, y_train_batch

def image_set_preprocessing_v2(X, y, batch_ratio = 1, pad_size=2, pad_method='constant'):
    X_pad = np.pad(X, ((0,0),(pad_size,pad_size),(pad_size,pad_size)), 'constant')
    
    batch_size = (int)(X_pad.shape[0]*batch_ratio)
    
    order = np.array(range(X_pad.shape[0]))
    np.random.shuffle(order)
    X_pad_shuffle = X_pad[order]
    y_shuffle = y[order]

    X_train_batch = ((X_pad_shuffle[0:batch_size, :, :]).astype('float32'))/255
    for i in range(X_train_batch.shape[0]):
        X_train_batch[i,:,:] = random_noise(X_train_batch[i,:,:], clip=True)
    y_train_batch = y_shuffle[0:batch_size,]
    X_train_batch = X_train_batch.reshape(X_train_batch.shape[0], X_train_batch.shape[1], X_train_batch.shape[2], 1)
    
    return X_train_batch, y_train_batch

def flatten_patches(cubio, size):
    window_shape = (1,size,size,cubio.shape[3])
    step = (1,size,size,cubio.shape[3])
    patches = view_as_windows(cubio, window_shape, step)
    patches = patches.squeeze(axis = (3,4))
    patches_panel = patches.reshape(-1, patches.shape[-3]*patches.shape[-2]*patches.shape[-1])
    
    return patches_panel

def remove_low_variance(patches_panel,thr=0.05):
    var = np.var(patches_panel, axis = 1)
    patches_clean = patches_panel[var>thr]
    
    return patches_clean

def remove_patches_mean(patches):
    mean = patches.mean(axis = 1)
    patches_mean_remov = (patches.T-mean).T
    
    return patches_mean_remov

def pca_kernel(patches, n_comps, kernel):
    pca = PCA(n_components = n_comps)
    pca.fit(patches)
    kernel.append(pca)  
    
def pca_kernel_v2(patches, kernel):
    pca = PCA()
    pca.fit(patches)
    kernel.append(pca)
    
def pca_kernel_v3(patches, n_comps, kernel):
    pca = KernelPCA(n_components = n_comps)
    pca.fit(patches)
    kernel.append(pca)
    
def pca_transform(kernel, layer, patches_mean_remov, patches_panel, X_train):
    n_sample = X_train.shape[0]
    h = (int)(X_train.shape[1]/pow(2,layer))
    patches_proj = k[layer-1].transform(patches_mean_remov)
    cubio_pos = patches_proj.reshape(n_sample,h,h,-1)
    cubio_neg = -cubio_pos
    cubio_pca = np.concatenate((cubio_pos, cubio_neg), axis=3)
    dc = patches_panel.mean(axis=1)*2
    dc = dc.reshape(n_sample, h, h, -1)
    cubio_next = np.concatenate((cubio_pca, dc), axis=3)
    
    return cubio_next

def pca_transform_v2(kernel, layer, patches_mean_remov, patches_panel, X_train):
    n_sample = X_train.shape[0]
    h = (int)(X_train.shape[1]/pow(2,layer))
    patches_proj = k[layer-1].transform(patches_mean_remov)
    cubio_pos = patches_proj.reshape(n_sample,h,h,-1)
    dc = patches_panel.mean(axis=1)*2
    dc = dc.reshape(n_sample, h, h, -1)
    cubio_next = np.concatenate((cubio_pos, dc), axis=3)
    
    return cubio_next

def pca_transform_complete(kernel, layer, patches_mean_remov, patches_panel, X_train):
    n_sample = X_train.shape[0]
    h = (int)(X_train.shape[1]/pow(2,layer))
    w = pow(2,layer)
    patches_proj = k[layer-1].transform(patches_mean_remov)
    cubio_pos = patches_proj.reshape(n_sample,h,h,w,w,-1)
    Swap_cubio = np.swapaxes(cubio_pos, 2, 3)
    cubio_pos = Swap_cubio.reshape(n_sample,32,32,-1)
    cubio_neg = -cubio_pos
    cubio_pca = np.concatenate((cubio_pos, cubio_neg), axis=3)
    
    return cubio_pca

def dc_add(X_train, layer, cubio, cubio_pca):
    n_sample = 60000
    cubio_enlarge = np.zeros((n_sample,32*pow(2,1),32*pow(2,1),cubio.shape[3]))
    for i in range(n_sample):
        cubio_enlarge[i,:,:,:] = resize(cubio[i,:,:,:], output_shape=[cubio.shape[1]*2,cubio.shape[2]*2,cubio.shape[3]])
    window_shape = (1,2,2,cubio_enlarge.shape[3])
    step = (1,2,2,cubio_enlarge.shape[3])
    patches = view_as_windows(cubio_enlarge, window_shape, step)
    patches = patches.squeeze(axis = (3,4))
    patches_panel = patches.reshape(-1, patches.shape[-3]*patches.shape[-2]*patches.shape[-1])

    dc = patches_panel.mean(axis=1)*2
    dc = dc.reshape(n_sample, 32, 32, -1)
    cubio_next = np.concatenate((cubio_pca, dc), axis=3)
    
    return cubio_next

def relu(cubio):
    cubio_relu = cubio * (cubio > 0)
    
    return cubio_relu

def tanh(cubio):
    cubio_tanh = np.divide(np.exp(cubio), 1+np.exp(cubio))
    
    return cubio_tanh
    
def complete_base_training(cubio, layer, n_comps, kernel, feature_list, X):
    print("training at the %dth layer ..." %(layer))
    patches_panel = flatten_patches(cubio,pow(2,layer))
    patches_mean_remov = remove_patches_mean(patches_panel)
    if layer == 1:
        patches_clean = remove_low_variance(patches_mean_remov)
    else:
        patches_clean = patches_mean_remov
    pca_kernel(patches_clean,patches_clean.shape[1], kernel)
    cubio_pca = relu(pca_transform_complete(kernel, layer, patches_mean_remov, patches_panel, X))
    cubio_next = dc_add(X, layer, cubio, cubio_pca)
    #feature_list.append(cubio_next)
    print("Done! the shape of output cubio is %s." %(cubio_next.shape,))
    
    return cubio_next   
    
def one_stage_training(layer, n_comps, kernel, feature_list, X):
    print("training at the %dth layer ..." %(layer))
    patches_panel = flatten_patches(X, pow(2,layer))
    patches_mean_remov = remove_patches_mean(patches_panel)
    if layer == 1:
        patches_clean = remove_low_variance(patches_mean_remov)
    else:
        patches_clean = patches_mean_remov
    pca_kernel(patches_clean, n_comps[layer-1], k)
    cubio_next = relu(pca_transform(k, layer, patches_mean_remov, patches_panel, X_train))
    feature_list.append(cubio_next)
    print("Done! the shape of output cubio is %s." %(cubio_next.shape,))

def feature_fusion(feature_list, num_layers):
    feature = feature_list[0].reshape(feature_list[0].shape[0], -1)
    for i in range(num_layers-1):
        feature = np.concatenate((feature,feature_list[i+1].reshape(feature_list[i+1].shape[0], -1)), axis=1)
    print("the shape of features we get is %s." %(feature.shape,))
    return feature

def Reduce_Feature(n_comps, feature):
    pca = PCA(n_components = n_comps)
    X_pc = pca.fit_transform(feature)
    print("the number of dimensions kept is %d." %(X_pc.shape[1]))
    
    return X_pc, pca

def Reduce_Feature_v2(n_comps, feature):
    pca = KernelPCA(n_components = n_comps)
    X_pc = pca.fit_transform(feature)
    print("the number of dimensions kept is %d." %(X_pc.shape[1]))
    
    return X_pc, pca

def F_test(percent, feature, label):
    Ftest = SelectPercentile(chi2, percent)
    X_f = Ftest.fit_transform(feature, label)
    print("the number of feature dimensions passing F-test is %d." %(X_f.shape[1]))

    return X_f, Ftest

def RandomForest_FeatureSelect(percent, feature, label):
    forest = RandomForestClassifier(random_state=0, )
    forest.fit(feature, label)
    index = np.argsort(forest.feature_importances_*100)
    num = (int)((float)(feature.shape[1]*percent)/100)
    X_rf = feature[:,index[0:num]]
    print("the number of feature dimensions passing Random-Forest feature selection is %d." %(X_rf.shape[1]))
    
    return X_rf, forest, index

def RandomForest_FeatureSelect_test(percent, feature, index):
    num = (int)((float)(feature.shape[1]*percent)/100)
    X_rf = feature[:,index[0:num]]
    print("the number of feature dimensions passing Random-Forest feature selection is %d." %(X_rf.shape[1]))
    
    return X_rf
    
def SVM_training(feature, label, n_comps, percent):
    print('SVM is under training...')
    X_f, Ftest = F_test(percent, feature, label)
    X_pc, pca = Reduce_Feature(n_comps, X_f)
    clf = SVC()
    clf.fit(X_pc, label)
    y_pred = clf.predict(X_pc)
    accuracy = accuracy_score(label, y_pred)
    print("SVM accuracy on training sample is %f" %(accuracy))
    
    return Ftest, pca, clf, accuracy

def RF_training(feature, label, n_comps, percent):
    print('Random Forest is under training...')
    X_f, Ftest = F_test(percent, feature, label)
    X_pc, pca = Reduce_Feature(n_comps, X_f)
    clf = RandomForestClassifier()  
    clf.fit(X_pc, label)
    y_pred = clf.predict(X_pc)
    accuracy = accuracy_score(label, y_pred)
    print("RF accuracy on training sample is %f" %(accuracy))
    
    return Ftest, pca, clf, accuracy

def SVM_training_v2(X_pc, label):
    print('SVM is under training...')
    clf = SVC()
    clf.fit(X_pc, label)
    y_pred = clf.predict(X_pc)
    accuracy = accuracy_score(label, y_pred)
    print("SVM accuracy on training sample is %f" %(accuracy))
    
    return clf, accuracy

def one_stage_testing(layer, kernel, feature_list, X):
    print("training at the %dth layer ..." %(layer))
    patches_panel = flatten_patches(X, pow(2,layer))
    patches_mean_remov = remove_patches_mean(patches_panel)
    cubio_next = relu(pca_transform(kernel, layer, patches_mean_remov, patches_panel, X))
    feature_list.append(cubio_next)
    print("Done! the shape of output cubio is %s." %(cubio_next.shape,))

def RF_testing(feature, label, Ftest, pca, clf):
    print("Test smaples are under Random Forest's testing...")
    feature_test = Ftest.transform(feature)
    print("the number of feature dimensions passing F-test is %d." %(feature_test.shape[1]))
    feature_pca = pca.transform(feature_test)
    print("the number of dimensions kept is %d." %(feature_pca.shape[1]))
    y_pred = clf.predict(feature_pca)
    accuracy = accuracy_score(label, y_pred)     
    print("RF accuracy on test sample is %f" %(accuracy))
    
    return accuracy, y_pred
          
def SVM_testing(feature, label, Ftest, pca, clf):
    print("Test smaples are under SVM's testing...")
    feature_test = Ftest.transform(feature)
    print("the number of feature dimensions passing F-test is %d." %(feature_test.shape[1]))
    feature_pca = pca.transform(feature_test)
    print("the number of dimensions kept is %d." %(feature_pca.shape[1]))
    y_pred = clf.predict(feature_pca)
    accuracy = accuracy_score(label, y_pred)     
    print("SVM accuracy on test sample is %f" %(accuracy))
    
    return accuracy, y_pred

# Modified Saak transform for image reconstruction

In [3]:
#prepare training parameters list
k = []
feature_list = []
n_comps = [3, 4, 7, 6, 8]

#preprocess input image batch and label batch
X_train, y_train_batch = image_set_preprocessing(X_train_raw, y_train_raw, batch_ratio = 1)
cubio = copy.deepcopy(X_train)

#compute the number of layers
layer_cnt = (int)(log(X_train.shape[1])/log(2))

X_train.shape

(60000, 32, 32, 1)

In [4]:
for i in range(layer_cnt):
    layer = i + 1
    one_stage_training(layer, n_comps, k, feature_list, X_train)

training at the 1th layer ...
Done! the shape of output cubio is (60000, 16, 16, 7).
training at the 2th layer ...
Done! the shape of output cubio is (60000, 8, 8, 9).
training at the 3th layer ...
Done! the shape of output cubio is (60000, 4, 4, 15).
training at the 4th layer ...
Done! the shape of output cubio is (60000, 2, 2, 13).
training at the 5th layer ...
Done! the shape of output cubio is (60000, 1, 1, 17).


In [5]:
#colloect all the features from all layers
features = feature_fusion(feature_list, layer_cnt)
features.shape

the shape of features we get is (60000, 2677).


(60000, 2677)

In [7]:
features.max()

8.129612

In [64]:
#feature selection
X_f, Ftest = F_test(50, features, y_train_batch)

#feature reduction
X_pc_32, pca_32 = Reduce_Feature(32, X_f)
X_pc_64, pca_64 = Reduce_Feature(64, X_f)
X_pc_128, pca_128 = Reduce_Feature(128, X_f)

#SVM training
svm_32, accuracy_svm_training_32= SVM_training_v2(X_pc_32, y_train_batch)
svm_64, accuracy_svm_training_64= SVM_training_v2(X_pc_64, y_train_batch)
svm_128, accuracy_svm_training_128= SVM_training_v2(X_pc_128, y_train_batch)

the number of feature dimensions passing F-test is 1338.
the number of dimensions kept is 32.
the number of dimensions kept is 64.
the number of dimensions kept is 128.
SVM is under training...
SVM accuracy on training sample is 0.996350
SVM is under training...
SVM accuracy on training sample is 0.995067
SVM is under training...
SVM accuracy on training sample is 0.991517


In [66]:
#preprocess testing image batch and label batch 
X_test, y_test_batch = image_set_preprocessing(X_test_raw, y_test_raw, batch_ratio = 1)

#prepare testing parameters list
feature_list_test = []

#forward testing process
for i in range(layer_cnt):
    layer = i + 1
    one_stage_testing(layer, k, feature_list_test, X_test)

training at the 1th layer ...
Done! the shape of output cubio is (10000, 16, 16, 7).
training at the 2th layer ...
Done! the shape of output cubio is (10000, 8, 8, 9).
training at the 3th layer ...
Done! the shape of output cubio is (10000, 4, 4, 15).
training at the 4th layer ...
Done! the shape of output cubio is (10000, 2, 2, 13).
training at the 5th layer ...
Done! the shape of output cubio is (10000, 1, 1, 17).


In [67]:
#colloect all the features from all layers
features_test = feature_fusion(feature_list_test, layer_cnt)
features_test.shape

#F-test --> PCA reducing dims --> SVM testing
accuracy_svm_testing_32, y_pred_svm_32 = SVM_testing(features_test, y_test_batch, Ftest, pca_32, svm_32)
accuracy_svm_testing_64, y_pred_svm_64 = SVM_testing(features_test, y_test_batch, Ftest, pca_64, svm_64)
accuracy_svm_testing_128, y_pred_svm_128 = SVM_testing(features_test, y_test_batch, Ftest, pca_128, svm_128)

the shape of features we get is (10000, 2677).
Test smaples are under SVM's testing...
the number of feature dimensions passing F-test is 1338.
the number of dimensions kept is 32.
SVM accuracy on test sample is 0.982900
Test smaples are under SVM's testing...
the number of feature dimensions passing F-test is 1338.
the number of dimensions kept is 64.
SVM accuracy on test sample is 0.985600
Test smaples are under SVM's testing...
the number of feature dimensions passing F-test is 1338.
the number of dimensions kept is 128.
SVM accuracy on test sample is 0.983200
